In [1]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_columns', 50)  # Показывать до 50 колонок
pd.set_option('display.max_colwidth', 100)  # Показывать до 50 колонок


data = pd.read_excel("../cuprum_3.xlsx", sheet_name="Лист4") #cuprum/Лист4/Лист2/Лист1
data.rename(columns={"пол":"gender", "возраст":"age"}, inplace=True)
data = data[data['action_type'] == 'CLICKED']
#data = data[data['tags'] != 'Секс']
data.drop(columns=["esb_ehr_id","patientnet_ehr_id", "medialog_ehr_id","action_type"], inplace=True)
data


# оставляем только тех пользователей, у которых есть не слишком много и не слишком мало взаимодействий
def filter_for_iteration_range(df_raw, min=4, max=50):
    interaction_counts = df_raw.groupby('ehr_id')['article_id'].count()
    users_in_range = set(interaction_counts[(interaction_counts >= min) & (interaction_counts <= max)].index)
    df_filtered = data[data['ehr_id'].isin(users_in_range)]
    return df_filtered

# негерируем негативные сэмлы для работы модели
def generate_negative_samples(df, all_articles, n_negatives=3):
    negatives = []

    for user_id, group in df.groupby('ehr_id'):
        clicked = set(group['article_id'])
        not_clicked = list(all_articles - clicked)
        sampled = np.random.choice(not_clicked, size=n_negatives, replace=False)

        for art in sampled:
            negatives.append({
                "ehr_id": user_id,
                "article_id": art,
                "label": 0
            })

    return pd.DataFrame(negatives)

data = filter_for_iteration_range(data)
data

,id,article_id,ehr_id,created_at,gender,age,birthday,title,url,views,published_date,formats,tags,rubric_title
44,3753815,60258f50e3c78375686c0c51,1132756,2023-07-01,1,48,1976-12-22,"​Как понять, что у вас паразиты",https://cuprum.media/science-answers/find-parasites?from=avaapp,5779.0,2021-02-11,longread,0,Ответили по науке
51,3754559,62e24b5be4ff39162208fd33,1169819,2023-07-01,1,31,1993-07-08,Как скрининг помог вовремя обнаружить дисплазию шейки матки,https://cuprum.media/interview/neoplasia-screening?from=avaapp,4580.0,2022-07-28,interview,"Диагностика,Женское здоровье",Интервью
52,3754575,62e24b5be4ff39162208fd33,1169819,2023-07-01,1,31,1993-07-08,Как скрининг помог вовремя обнаружить дисплазию шейки матки,https://cuprum.media/interview/neoplasia-screening?from=avaapp,4580.0,2022-07-28,interview,"Диагностика,Женское здоровье",Интервью
53,3754576,62e24610a9c9279df00b8673,1169819,2023-07-01,1,31,1993-07-08,"Что почитать: «Мальчик, который не переставал расти» Эдвина Кёрка",https://cuprum.media/razbor/the-boy-who-never-stopped-growing?from=avaapp,347.0,2022-07-28,longread,Книги,Разбор
54,3754581,62e0f1c5df18e4454c047292,1169819,2023-07-01,1,31,1993-07-08,«В суете нашей жизни некогда подумать о собственном здоровье»: как предотвратить инсульт,https://cuprum.media/interview/stroke-prevention?from=avaapp,892.0,2022-07-27,interview,Диагностика,Интервью
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30815,9603468,622b44d60e7b1e54fe5e5342,272451,2024-07-30,1,52,1973-01-09,Cкрининг на рак груди,https://cuprum.media/spravochnik/screening-breast-cancer-sp?from=avaapp,1072.0,2022-03-11,shortread,Скрининг,Справочник
30816,9603700,6218dd9140e8460b6d1e326b,57714,2024-07-30,1,58,1966-06-25,"Если хорошенько потрястись, это поможет снять стресс?",https://cuprum.media/proverka-sluha/shaking-off?from=avaapp,6086.0,2022-02-25,longread,Ментальное,Проверка слуха
30817,9603737,617bfecbd4f2793d703e2db2,1145846,2024-07-30,1,36,1989-05-11,"Одноразовые электронные сигареты опаснее, чем обычные вейпы?",https://cuprum.media/razbor/vape-nation?from=avaapp,11716.0,2021-10-29,longread,"Горло,Гаджеты",Разбор
30818,9603851,63e10773dbd23b1bff0b7002,263328,2024-07-30,1,31,1993-09-13,Плесень на орехах и кукурузе вызывает рак печени?,https://cuprum.media/science-answers/nuts-and-cancer?from=avaapp,1455.0,2023-02-06,longread,"Еда,Онкология,Патогены",Ответили по науке


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

class ContentRecommenderSystem:
    """
    Контентная рекомендательная система на основе TF-IDF и косинусного сходства
    """
    
    def __init__(self, df):
        """
        Инициализация системы рекомендаций
        
        Parameters:
        df: DataFrame с данными о статьях и пользователях
        """
        self.df = df.copy()
        self.tfidf_matrix = None
        self.article_features = None
        self.prepare_data()
        
    def prepare_data(self):
        """Подготовка данных для рекомендательной системы"""
        
        # Удаляем дубликаты статей для создания каталога
        self.articles = self.df[['article_id', 'title', 'tags', 'rubric_title', 'views', 'published_date']].drop_duplicates('article_id')
        
        # Заполняем пропущенные значения
        self.articles['tags'] = self.articles['tags'].fillna('')
        self.articles['rubric_title'] = self.articles['rubric_title'].fillna('')
        self.articles['title'] = self.articles['title'].fillna('')
        
        # Создаем комбинированное текстовое представление статьи
        # Даем больший вес заголовку, повторяя его
        self.articles['content'] = (
            self.articles['title'] + ' ' + 
            self.articles['title'] + ' ' +  # удваиваем заголовок для большего веса
            self.articles['tags'].astype(str).str.replace(',', ' ') + ' ' + 
            self.articles['rubric_title']
        )
        
        # Создаем TF-IDF матрицу
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            ngram_range=(1, 2),  # используем униграммы и биграммы
            min_df=2,
            max_df=0.8,
            token_pattern=r'[а-яА-Яa-zA-Z]+' # учитываем и русские, и английские слова
        )
        
        self.tfidf_matrix = self.vectorizer.fit_transform(self.articles['content'])
        
        # Добавляем нормализованную популярность как дополнительный признак
        scaler = MinMaxScaler()
        popularity_scores = scaler.fit_transform(self.articles[['views']].fillna(0))
        
        # Комбинируем TF-IDF с популярностью (90% контент, 10% популярность)
        self.content_similarity = cosine_similarity(self.tfidf_matrix)
        self.final_similarity = 0.9 * self.content_similarity + 0.1 * cosine_similarity(popularity_scores)
        
        print(f"Подготовлено {len(self.articles)} уникальных статей")
        print(f"Размер TF-IDF матрицы: {self.tfidf_matrix.shape}")
        
    def get_user_history(self, user_id):
        """
        Получение истории просмотров пользователя
        
        Parameters:
        user_id: ID пользователя (ehr_id)
        
        Returns:
        list: список ID просмотренных статей
        """
        user_articles = self.df[self.df['ehr_id'] == user_id]['article_id'].unique()
        return user_articles
    
    def get_similar_articles(self, article_id, top_n=10):
        """
        Получение похожих статей для данной статьи
        
        Parameters:
        article_id: ID статьи
        top_n: количество рекомендаций
        
        Returns:
        DataFrame с рекомендациями
        """
        if article_id not in self.articles['article_id'].values:
            return pd.DataFrame()
        
        # Находим индекс статьи
        idx = self.articles[self.articles['article_id'] == article_id].index[0]
        article_idx = self.articles.index.get_loc(idx)
        
        # Получаем похожие статьи
        sim_scores = list(enumerate(self.final_similarity[article_idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Исключаем саму статью (первая в списке)
        sim_scores = sim_scores[1:top_n+1]
        
        # Получаем индексы и scores
        article_indices = [i[0] for i in sim_scores]
        similarity_scores = [i[1] for i in sim_scores]
        
        # Создаем DataFrame с результатами
        recommendations = self.articles.iloc[article_indices][['article_id', 'title', 'tags', 'rubric_title']].copy()
        recommendations['similarity_score'] = similarity_scores
        
        return recommendations
    
    def recommend_for_user(self, user_id, top_n=10, method='weighted'):
        """
        Рекомендации для пользователя на основе истории просмотров
        
        Parameters:
        user_id: ID пользователя (ehr_id)
        top_n: количество рекомендаций
        method: метод агрегации ('weighted' - взвешенный, 'max' - максимум, 'avg' - среднее)
        
        Returns:
        DataFrame с рекомендациями
        """
        # Получаем историю пользователя
        user_history = self.get_user_history(user_id)
        
        if len(user_history) == 0:
            print(f"Пользователь {user_id} не найден или нет истории просмотров")
            return self.get_popular_articles(top_n)
        
        print(f"Пользователь {user_id} просмотрел {len(user_history)} статей")
        
        # Получаем все статьи для рекомендаций
        all_article_ids = self.articles['article_id'].values
        
        # Исключаем уже просмотренные
        candidate_articles = [a for a in all_article_ids if a not in user_history]
        
        # Словарь для хранения scores
        article_scores = {}
        
        # Для каждой просмотренной статьи находим похожие
        for viewed_article in user_history:
            if viewed_article not in self.articles['article_id'].values:
                continue
                
            # Находим индекс просмотренной статьи
            idx = self.articles[self.articles['article_id'] == viewed_article].index[0]
            article_idx = self.articles.index.get_loc(idx)
            
            # Получаем схожесть со всеми статьями
            similarities = self.final_similarity[article_idx]
            
            # Обновляем scores для кандидатов
            for i, candidate_id in enumerate(self.articles['article_id'].values):
                if candidate_id in candidate_articles:
                    if candidate_id not in article_scores:
                        article_scores[candidate_id] = []
                    article_scores[candidate_id].append(similarities[i])
        
        # Агрегируем scores в зависимости от метода
        final_scores = {}
        for article_id, scores in article_scores.items():
            if method == 'weighted':
                # Взвешенный score с учетом свежести просмотров
                weights = np.linspace(0.5, 1.0, len(scores))  # новые просмотры важнее
                final_scores[article_id] = np.average(scores, weights=weights[-len(scores):])
            elif method == 'max':
                final_scores[article_id] = max(scores)
            else:  # avg
                final_scores[article_id] = np.mean(scores)
        
        # Сортируем по score
        sorted_articles = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
        
        # Создаем DataFrame с рекомендациями
        rec_ids = [a[0] for a in sorted_articles]
        rec_scores = [a[1] for a in sorted_articles]
        
        recommendations = self.articles[self.articles['article_id'].isin(rec_ids)][
            ['article_id', 'title', 'tags', 'rubric_title', 'views']
        ].copy()
        
        # Добавляем scores
        recommendations['recommendation_score'] = recommendations['article_id'].map(
            dict(zip(rec_ids, rec_scores))
        )
        
        # Сортируем по score
        recommendations = recommendations.sort_values('recommendation_score', ascending=False)
        
        return recommendations
    
    def get_popular_articles(self, top_n=10):
        """
        Получение самых популярных статей (fallback для холодного старта)
        
        Parameters:
        top_n: количество статей
        
        Returns:
        DataFrame с популярными статьями
        """
        return self.articles.nlargest(top_n, 'views')[['article_id', 'title', 'tags', 'rubric_title', 'views']]
    
    def evaluate_diversity(self, recommendations):
        """
        Оценка разнообразия рекомендаций
        
        Parameters:
        recommendations: DataFrame с рекомендациями
        
        Returns:
        dict: метрики разнообразия
        """
        if len(recommendations) == 0:
            return {}
        
        # Уникальные рубрики
        unique_rubrics = recommendations['rubric_title'].nunique()
        
        # Среднее попарное расстояние между рекомендациями
        rec_indices = []
        for article_id in recommendations['article_id'].values:
            if article_id in self.articles['article_id'].values:
                idx = self.articles[self.articles['article_id'] == article_id].index[0]
                rec_indices.append(self.articles.index.get_loc(idx))
        
        if len(rec_indices) > 1:
            avg_distance = 0
            count = 0
            for i in range(len(rec_indices)):
                for j in range(i+1, len(rec_indices)):
                    avg_distance += 1 - self.content_similarity[rec_indices[i], rec_indices[j]]
                    count += 1
            avg_distance = avg_distance / count if count > 0 else 0
        else:
            avg_distance = 0
        
        return {
            'unique_rubrics': unique_rubrics,
            'rubrics_ratio': unique_rubrics / len(recommendations),
            'avg_content_distance': avg_distance
        }


# Пример использования
def demo_recommendations(df):
    """
    Демонстрация работы рекомендательной системы
    """
    # Создаем систему рекомендаций
    recommender = ContentRecommenderSystem(df)
    
    # Выбираем случайного пользователя с историей
    users_with_history = df.groupby('ehr_id').size()
    active_users = users_with_history[users_with_history >= 5].index.tolist()
    
    if active_users:
        sample_user = np.random.choice(active_users)
        
        print(f"\n{'='*50}")
        print(f"Рекомендации для пользователя {sample_user}")
        print(f"{'='*50}\n")
        
        # История пользователя
        user_history = recommender.get_user_history(sample_user)
        history_df = df[df['ehr_id'] == sample_user][['article_id', 'title']].drop_duplicates()
        
        print("История просмотров пользователя:")
        for _, row in history_df.head(5).iterrows():
            print(f"- {row['title'][:80]}...")
        
        # Получаем рекомендации
        recommendations = recommender.recommend_for_user(sample_user, top_n=10)
        
        print(f"\n{'='*50}")
        print("Топ-10 рекомендаций:")
        print(f"{'='*50}\n")
        
        for i, row in recommendations.iterrows():
            print(f"{len(recommendations) - recommendations.index.get_loc(i)}. {row['title'][:70]}...")
            print(f"   Рубрика: {row['rubric_title']}, Score: {row['recommendation_score']:.3f}")
            print()
        
        # Оценка разнообразия
        diversity = recommender.evaluate_diversity(recommendations)
        print(f"\n{'='*50}")
        print("Метрики разнообразия рекомендаций:")
        print(f"{'='*50}")
        print(f"Уникальных рубрик: {diversity['unique_rubrics']}")
        print(f"Доля уникальных рубрик: {diversity['rubrics_ratio']:.2%}")
        print(f"Среднее расстояние между рекомендациями: {diversity['avg_content_distance']:.3f}")
    
    return recommender


# Если у вас уже есть загруженный DataFrame df, используйте:
# recommender = ContentRecommenderSystem(df)
# recommendations = recommender.recommend_for_user(user_id=1169819, top_n=10)
# print(recommendations)

In [3]:
def generate_recommendations_for_all(df, output_path="recommendations.csv", top_n=10):
    # Инициализируем систему
    recommender = ContentRecommenderSystem(df)
    
    # Получаем список всех пользователей
    users = df['ehr_id'].unique()
    #users = df['ehr_id']
    #users = users.head(10)
    
    all_recs = []

    for user_id in users:
        recs = recommender.recommend_for_user(user_id, top_n=top_n)
        
        if recs is not None and len(recs) > 0:
            recs = recs.copy()
            recs['ehr_id'] = user_id
            all_recs.append(recs[['ehr_id', 'article_id', 'title', 'rubric_title', 'recommendation_score']])
    
    # Объединяем все рекомендации
    result_df = pd.concat(all_recs, ignore_index=True)
    
    # Сохраняем в CSV
    result_df.to_csv(output_path, index=False, encoding="utf-8")
    print(f"Сохранено {len(result_df)} рекомендаций для {len(users)} пользователей в файл {output_path}")

    return result_df


In [4]:
recs_df = generate_recommendations_for_all(data, "user_recommendations_all.csv", top_n=20)

Подготовлено 2954 уникальных статей
Размер TF-IDF матрицы: (2954, 1000)
Пользователь 1132756 просмотрел 5 статей
Пользователь 1169819 просмотрел 21 статей
Пользователь 966281 просмотрел 8 статей
Пользователь 780606 просмотрел 8 статей
Пользователь 232858 просмотрел 6 статей
Пользователь 1153297 просмотрел 4 статей
Пользователь 991913 просмотрел 12 статей
Пользователь 1241112 просмотрел 4 статей
Пользователь 1173385 просмотрел 10 статей
Пользователь 1273528 просмотрел 5 статей
Пользователь 1278017 просмотрел 5 статей
Пользователь 234386 просмотрел 9 статей
Пользователь 390291 просмотрел 5 статей
Пользователь 933336 просмотрел 11 статей
Пользователь 1031865 просмотрел 8 статей
Пользователь 541168 просмотрел 4 статей
Пользователь 511237 просмотрел 5 статей
Пользователь 381190 просмотрел 5 статей
Пользователь 1082313 просмотрел 5 статей
Пользователь 1278693 просмотрел 8 статей
Пользователь 1292915 просмотрел 17 статей
Пользователь 439638 просмотрел 4 статей
Пользователь 1244233 просмотрел 